In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../")


from sae.train import ModelTrainer
from sae.config import create_config, Config
from sae.utils import get_blog_checkpoint, get_blog_sparsity, create_lineplot_histogram

from unlearning.metrics import calculate_wmdp_bio_metrics, calculate_wmdp_bio_metrics_hf, get_loss_added_rmu_model
from unlearning.tool import get_basic_gemma_2b_it_layer9_act_store
from unlearning.var import gemma_2b_it_rmu_model_names
from unlearning.metrics import all_permutations, calculate_control_metrics

from transformer_lens import HookedTransformer, utils
from sae.metrics import compute_metrics_post_by_text


import plotly.express as px
import plotly.graph_objs as go
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformer_lens import HookedTransformer
import time

import torch

torch.set_grad_enabled(False)

In [2]:
print("Getting Hugging Face model")

hf_model_name = "google/gemma-2b-it"
tokenizer = AutoTokenizer.from_pretrained(hf_model_name)
hf_model = AutoModelForCausalLM.from_pretrained(hf_model_name, torch_dtype='auto')

transformer_lens_model_name = "google/gemma-2b-it"
base_model = HookedTransformer.from_pretrained(transformer_lens_model_name, hf_model=hf_model, tokenizer=tokenizer)

print("done")

Getting Hugging Face model


`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded pretrained model google/gemma-2b-it into HookedTransformer
done


In [3]:
# metrics_list = []
# loss_added_list = []
control_metrics_list = []

# model_names = ['gemma_2b_it_rmu_6', 'gemma_2b_it_rmu_10', 'gemma_2b_it_rmu_30', 'gemma_2b_it_rmu_60', 'gemma_2b_it_rmu_100']

for hf_model_name in gemma_2b_it_rmu_model_names[3:4]:

    # hf_model_name = "eoinf/" + model_name
    hf_model = AutoModelForCausalLM.from_pretrained(hf_model_name, torch_dtype='auto') #.to("cuda")

    rmu_model = HookedTransformer.from_pretrained(transformer_lens_model_name, hf_model=hf_model, tokenizer=tokenizer)
    dataset_name = "high_school_us_history"
    control_metrics = calculate_control_metrics(rmu_model, dataset_name, random_select_one=False, question_subset=None, permutations=[[0, 1, 2, 3]])
    control_metrics_list.append(control_metrics)
    # metrics = calculate_wmdp_bio_metrics(rmu_model,
    #                                     question_subset=None,
    #                                     question_subset_file="../data/wmdp-bio_gemma_2b_it_correct.csv",
    #                                     permutations=None)
    
    # metrics_list.append(metrics)
    
    # print("done metrics")
    
    # loss_added = get_loss_added_rmu_model(rmu_model, base_model, activation_store, n_batch=20)
    
    # loss_added_list.append(loss_added)
    # print("done", hf_model_name)

print("done")

config.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

Loaded pretrained model google/gemma-2b-it into HookedTransformer


NameError: name 'model' is not defined

In [ ]:
activation_store = get_basic_gemma_2b_it_layer9_act_store(base_model)

buffer
dataloader


In [ ]:
metrics_list = []
loss_added_list = []

# model_names = ['gemma_2b_it_rmu_6', 'gemma_2b_it_rmu_10', 'gemma_2b_it_rmu_30', 'gemma_2b_it_rmu_60', 'gemma_2b_it_rmu_100']

for hf_model_name in gemma_2b_it_rmu_model_names:

    # hf_model_name = "eoinf/" + model_name
    hf_model = AutoModelForCausalLM.from_pretrained(hf_model_name, torch_dtype='auto') #.to("cuda")

    rmu_model = HookedTransformer.from_pretrained(transformer_lens_model_name, hf_model=hf_model, tokenizer=tokenizer)
    
    metrics = calculate_wmdp_bio_metrics(rmu_model,
                                        question_subset=None,
                                        question_subset_file="../data/wmdp-bio_gemma_2b_it_correct.csv",
                                        permutations=None)
    
    metrics_list.append(metrics)
    
    print("done metrics")
    
    loss_added = get_loss_added_rmu_model(rmu_model, base_model, activation_store, n_batch=20)
    
    loss_added_list.append(loss_added)
    print("done", hf_model_name)
    
print("done")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded pretrained model google/gemma-2b-it into HookedTransformer


100%|██████████| 29/29 [00:07<00:00,  4.02it/s]


done metrics


100%|██████████| 20/20 [00:24<00:00,  1.21s/it]


...

Loaded pretrained model google/gemma-2b-it into HookedTransformer


100%|██████████| 29/29 [00:06<00:00,  4.18it/s]


done metrics


100%|██████████| 20/20 [00:24<00:00,  1.23s/it]


done eoinf/gemma_2b_it_rmu_100
done


In [ ]:
loss_added_list2 = []

# model_names = ['gemma_2b_it_rmu_6', 'gemma_2b_it_rmu_10', 'gemma_2b_it_rmu_30', 'gemma_2b_it_rmu_60', 'gemma_2b_it_rmu_100']

for hf_model_name in gemma_2b_it_rmu_model_names:

    hf_model = AutoModelForCausalLM.from_pretrained(hf_model_name, torch_dtype='auto') #.to("cuda")
    rmu_model = HookedTransformer.from_pretrained(transformer_lens_model_name, hf_model=hf_model, tokenizer=tokenizer)
    
    loss_added = get_loss_added_rmu_model(rmu_model, base_model, activation_store, n_batch=80)
    loss_added_list2.append(loss_added)
    
    print("done", hf_model_name)
    

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded pretrained model google/gemma-2b-it into HookedTransformer


100%|██████████| 80/80 [01:37<00:00,  1.22s/it]


done eoinf/gemma_2b_it_rmu_6


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

done eoinf/gemma_2b_it_rmu_60


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded pretrained model google/gemma-2b-it into HookedTransformer


100%|██████████| 80/80 [01:39<00:00,  1.24s/it]


done eoinf/gemma_2b_it_rmu_100
